In [ ]:
#概要
BNPパリバ・カーディフは個人保険の世界的なスペシャリストとして、 
ヨーロッパ、アジア、ラテンアメリカの36か国で9,000万人の顧客にサービスを提供しています。

新たな用途やライフスタイルの出現によって形作られた世界では、あらゆるものがますます加速しています。
予期せぬ事態に直面した際、お客様は保険会社に可能な限り迅速なサポートを期待します。
しかし、保険金請求処理では、請求が承認され、保険金が支払われるまでに、異なるレベルの審査が必要になる場合があります。
デジタル経済によって生み出された新しい習慣や行動様式に伴い、このプロセスはデータサイエンスを活用して適応し、
お客様の新たなニーズと期待に応える必要があります。

このチャレンジでは、BNPパリバ・カーディフは次の2つのカテゴリーの請求を匿名化したデータベースを提供しています。

・承認を早めることで支払いを早めることができる請求
＿承認前に追加情報が必要となる請求
Kaggler は、プロセスの早い段階で利用可能な機能に基づいて請求のカテゴリを予測するという課題に取り組んでおり、
BNP Paribas Cardif が請求プロセスを加速し、顧客に優れたサービスを提供できるように支援しています。

In [ ]:
#評価
このコンペティションの評価指標は Log Lossである。

𝐿𝑜𝑔𝐿𝑜𝑠𝑠=−(1/𝑁)∑𝑖=(1/𝑁)(𝑦𝑖log(𝑝𝑖)+(1–𝑦𝑖)log(1–𝑝𝑖))

ここで、
𝑁はサンプルの数
𝑦𝑖 は実際のクラスラベル（0または1）
𝑝𝑖 はモデルが予測したクラス1である確率（0から1の間の値）

#参考（https://qiita.com/exp/items/1c6c9a3fae2d97bfa0c7）
Logarithmic Loss（対数損失）、またはクロスエントロピー損失は、分類モデルの性能を評価するための一般的な損失関数です。
分類モデルの性能を測る指標。(このLog lossへの)入力は0~1の確率の値をとる。
この値を最小化したい。完璧なモデルではLog lossが0になる。
予測値が正解ラベルから離れるほどLog lossは増加する。

#特徴
確率の評価
・Log Lossは、モデルが出力する確率を評価するため、単なるクラスの予測精度よりも詳細な評価が可能です。
・これは特に、確率の校正が重要な応用において有用です。
ペナルティの適用
・Log Lossは、予測が自信を持って間違っている場合（例：正しいクラスが0.01の確率で予測される場合）に大きなペナルティを課します。
・これにより、モデルが不確実性を適切に扱うことを促します。
連続的な損失関数
・Log Lossは連続的な損失関数であり、確率の小さな変動に対しても損失の変化を反映するため、モデルの微調整に対して敏感です。
モデルの比較
・Log Lossは異なるモデルの性能を比較するための標準的な指標として広く使用されます。
・特に異なるモデルが出力する確率の精度を評価する際に有効です。

#提出ファイル
テストデータセット内のすべての観測値について、提出ファイルにはIDとPredictedProbの2つの列を含める必要があります。ファイルにはヘッダーが含まれ、以下の形式である必要があります。

ID、予測確率
0、0.5 
1、0.5 
2、0.5 
7、0.5
など。

In [ ]:
#全体の流れ
1. ライブラリのインポート
   ↓
2. データの読み込み
   ↓
3. データ探索（EDA: Exploratory Data Analysis）
   - データの形、欠損値、分布を確認
   ↓
4. データ前処理
   - 欠損値の処理
   - 文字列データの数値化
   ↓
5. 特徴量エンジニアリング
   - 新しい特徴量の作成
   - 不要な特徴量の削除
   ↓
6. モデルの訓練
   - 訓練データを使ってモデルを学習
   ↓
7. 予測の生成
   - テストデータに対して予測を実行
   ↓
8. 提出ファイルの作成
   - Kaggle提出用のCSVファイルを作成

In [2]:
#ライブラリのインポート
## 必要なライブラリをインポート
import pandas as pd  # データ操作
import numpy as np   # 数値計算
import matplotlib.pyplot as plt  # グラフ描画
import seaborn as sns  # より見やすいグラフ
import warnings  # 警告を非表示にする

#EDA関連のライブラリ
from ydata_profiling import ProfileReport

## 機械学習関連のライブラリ
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

## 警告を非表示にする（見やすくするため）
warnings.filterwarnings('ignore')

## グラフのスタイル設定
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("ライブラリのインポートが完了しました！")

ライブラリのインポートが完了しました！


In [3]:
#データの読み込み
train_df = pd.read_csv('../input/bnp-paribas-cardif-claims-management/train.csv')
test_df = pd.read_csv('../input/bnp-paribas-cardif-claims-management/test.csv')

print("データの読み込みが完了しました！")

データの読み込みが完了しました！


In [6]:
#基本統計量の確認
train_df.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [7]:
#基本統計量の確認2
train_df.describe()

,ID,target,v1,v2,v4,v5,v6,v7,v8,v9,...,v121,v122,v123,v124,v126,v127,v128,v129,v130,v131
count,114321.000000,114321.000000,6.448900e+04,6.452500e+04,6.452500e+04,6.569700e+04,6.448900e+04,6.448900e+04,6.570200e+04,6.447000e+04,...,6.448100e+04,6.447000e+04,63643.000000,6.570200e+04,6.448900e+04,6.448900e+04,6.569700e+04,114321.000000,6.447800e+04,6.442600e+04
mean,114228.928228,0.761199,1.630686e+00,7.464411e+00,4.145098e+00,8.742359e+00,2.436402e+00,2.483921e+00,1.496569e+00,9.031859e+00,...,2.737596e+00,6.822439e+00,3.549938,9.198120e-01,1.672658e+00,3.239542e+00,2.030373e+00,0.310144,1.925763e+00,1.739389e+00
std,65934.487362,0.426353,1.082813e+00,2.961676e+00,1.148263e+00,2.036018e+00,5.999653e-01,5.894485e-01,2.783003e+00,1.930262e+00,...,1.356294e+00,1.795978e+00,2.604704,2.099407e+00,5.031683e-01,1.625988e+00,1.074232e+00,0.693262,1.264497e+00,1.134702e+00
min,3.000000,0.000000,-9.996497e-07,-9.817614e-07,-6.475929e-07,-5.287068e-07,-9.055091e-07,-9.468765e-07,-7.783778e-07,-9.828757e-07,...,-9.820642e-07,-9.978497e-07,0.019139,-9.994953e-07,-9.564174e-07,-9.223798e-07,8.197812e-07,0.000000,-9.901257e-07,-9.999134e-07
25%,57280.000000,1.000000,9.135798e-01,5.316428e+00,3.487398e+00,7.605918e+00,2.065064e+00,2.101477e+00,8.658986e-02,7.853659e+00,...,1.786965e+00,5.647712e+00,1.963315,2.053777e-02,1.417600e+00,2.101900e+00,1.393830e+00,0.000000,1.106172e+00,1.012658e+00
50%,114189.000000,1.000000,1.469550e+00,7.023803e+00,4.205991e+00,8.670867e+00,2.412790e+00,2.452166e+00,3.860317e-01,9.059582e+00,...,2.436195e+00,6.749117e+00,2.739239,1.398639e-01,1.614802e+00,2.963620e+00,1.798436e+00,0.000000,1.560138e+00,1.589403e+00
75%,171206.000000,1.000000,2.136128e+00,9.465497e+00,4.833250e+00,9.771353e+00,2.775285e+00,2.834285e+00,1.625246e+00,1.023256e+01,...,3.379175e+00,7.911392e+00,4.075361,8.718333e-01,1.843886e+00,4.108146e+00,2.390158e+00,0.000000,2.332425e+00,2.261905e+00
max,228713.000000,1.000000,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,...,2.000000e+01,2.000000e+01,19.686069,2.000000e+01,1.563161e+01,2.000000e+01,2.000000e+01,11.000000,2.000000e+01,2.000000e+01
